In [15]:
import os 
import sys 
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import re
import pickle
import json
import sqlite3
from pprint import pprint
import datetime
from tqdm.notebook import tqdm
tqdm.pandas()

/Users/malcolmtaylor/Github_Projects/venv/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [16]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth

In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
with open('data/spotify_creds.pkl', 'rb') as hnd:
    credentials = pickle.load(hnd)

In [19]:
os.environ.update(credentials)

In [20]:
scope = "user-read-recently-played"

In [21]:
os.environ['SPOTIPY_REDIRECT_URI'] = 'https://malcolm-taylor.info/'
spotify = spotipy.Spotify(client_credentials_manager= SpotifyOAuth(scope=scope
                                                                   , username='malchemist02'))

In [22]:
def split_utc_time_str(time_str):
    part1 = datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) - datetime.timedelta(hours=4)
    date = str(part1.date())
    time = str(part1.time().strftime('%H:%M'))
    return(date, time)

In [23]:
def get_recently_played(after=None):
    recently_played = spotify.current_user_recently_played(after)
    # Describe json 
    print('Getting songs at ', datetime.datetime.now())
    before = recently_played['cursors']['before']
    after = recently_played['cursors']['after']
    n_items = len(recently_played['items'])
    print(f'There are {n_items} songs between '
          , datetime.datetime.fromtimestamp(int(before)/1000)
          , '  and  '
          , datetime.datetime.fromtimestamp(int(after)/1000))
    
    if n_items == 0:
        return()
    
    # Parse recently played output 
    songs_list = []
    wanted_keys = ['name', 'duration_ms', 'popularity', 'uri']
    for song in recently_played['items']:
        temp_song_dict = {}
        temp_song_dict['played_at_date'], temp_song_dict['played_at_time'] = split_utc_time_str(song['played_at'])
        for k in wanted_keys:
            temp_song_dict[k] = song['track'][k]
        
        # Take first artist information 
        temp_song_dict['artist_name'] = song['track']['artists'][0]['name']
        temp_song_dict['artist_id'] = song['track']['artists'][0]['uri']
        temp_song_dict['playlist_id'] = song['context']['uri']
        
        songs_list.append(temp_song_dict)
    
    # Export cleanly 
    out_series = pd.DataFrame(songs_list)
    out_series['duration_min'] = out_series['duration_ms']/60000.0
    out_series = out_series.rename({'uri':'song_uri'}, axis=1) 
    out_series = out_series[['name', 'artist_name', 'played_at_date', 'played_at_time'
                            , 'duration_min', 'popularity'
                            , 'song_uri', 'artist_id', 'playlist_id']]
    
    return(out_series)

In [24]:
newly_played = get_recently_played()

KeyboardInterrupt: 

In [ ]:
newly_played.head()

In [25]:
recently_played = spotify.current_user_recently_played()
print(datetime.datetime.now())

KeyboardInterrupt: 

In [ ]:
recently_played

In [ ]:
wanted_keys = ['name', 'duration_ms', 'popularity', 'uri', 'id']
out1 = {}
for song in recently_played['items']:
    for k in wanted_keys:
        out1[k] = song['track'][k]
    out1['artist_name'] = song['track']['artists'][0]['name']
    out1['artist_id'] = song['track']['artists'][0]['uri']
    out1['playlist_id'] = song['context']['uri']
    out1['played_at'] = song['played_at']

In [ ]:
out1

In [ ]:
recently_played['items'][0].keys()

In [ ]:
time1 = recently_played['items'][0]['played_at']
time1

In [ ]:
datetime.datetime.fromisoformat(time1.replace('Z', '+00:00')) - datetime.timedelta(hours=4)

In [ ]:
split_utc_time_str(time1)

In [ ]:
time1.split('T')[1][:5]

In [ ]:
recently_played

In [ ]:
len(recently_played['items'])

In [ ]:
recently_played['items'][0].keys()

In [ ]:
(recently_played['items'][0]['played_at'])

In [ ]:
recently_played['items'][0]['track'].keys()

In [ ]:
track_keep_keys = ['name', 'artists', 'duration_ms', 'popularity', 'uri', 'id']

In [ ]:
recently_played['items'][0]['track']

In [ ]:
recently_played = spotify.current_user_recently_played()


In [ ]:
recently_played['items'][-1]['played_at']

In [ ]:
before = recently_played['cursors']['before']
print('before', before, datetime.datetime.fromtimestamp(int(before)/1000))
after = recently_played['cursors']['after']
print('after', after, datetime.datetime.fromtimestamp(int(after)/1000))

In [ ]:
recently_played['next']

In [ ]:
recently_played = spotify.current_user_recently_played(before = before)

In [ ]:
recently_played['items']

In [ ]:
playlists = spotify.user_playlists('malchemist02')
playlists

In [ ]:
len(playlists)

In [ ]:
names = [x['name'] for x in playlists['items']]
names

In [ ]:
def get_playlist_audio_feats(inp_name, playlists):
    pl_names = [x['name'] for x in playlists['items']]
    one_pl = [x for x in playlists['items'] if x['name'].lower() == inp_name.lower()]
    if len(one_pl) != 1:
        print(f"There are {len(one_pl)} playlists found")
        print(one_pl)
        print('Try again')
        return()
    playlist_uri = one_pl[0]['uri']
    tracks = spotify.playlist_tracks(playlist_uri)
    print(f"There are {len(tracks['items'])} songs in the playlist {inp_name}")
    uris = [x['track']['uri'] for x in tracks['items']]
    song_names = [x['track']['name'] for x in tracks['items']]
    track_audio_feats = spotify.audio_features(uris)
    pd_tracks = pd.DataFrame(track_audio_feats, index=song_names)
    pd_tracks['duration_s'] = pd_tracks['duration_ms']/1000.0
    pd_tracks = pd_tracks.drop(['duration_ms'], axis=1)
    return(pd_tracks)

In [ ]:
name1 = 'Malcolm Bachata'
playlist1 = get_playlist_audio_feats(name1, playlists)
name2 = 'malcolm salsa'
playlist2 = get_playlist_audio_feats(name2, playlists)

In [ ]:
playlist1.mean()

In [ ]:
playlist2.mean()

In [ ]:
(playlist1.mean() - playlist2.mean()).sort_values()

## Segmentation

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.tree import export_text
import matplotlib.pyplot as plt 
import graphviz

In [ ]:
### df1 = playlist1.copy()
df1['target'] = 'slow_salsa'
df2 = playlist2.copy()
df2['target'] = 'marc anthony'
df_total = pd.concat([df1, df2])
df_total = df_total[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_s', 'target']]
df_total.shape

In [ ]:
df_total.columns

In [ ]:
dt = DecisionTreeClassifier(max_depth=2)

In [ ]:
X = df_total.copy()
X = X.drop(['target'], axis=1)
y = df_total['target']

In [ ]:
dt.fit(X, y)

In [ ]:
tree.plot_tree(dt
               , feature_names=list(X.columns)
               , class_names=list(y.unique())) 


In [ ]:
r = export_text(dt, feature_names=list(X.columns))
print(r)

## Graveyard